In [1]:
import pyaudio
import wave
import os

def record_audio(filename, duration=5, sample_rate=44100):
    chunk = 1024  # Record in chunks of 1024 samples
    audio = pyaudio.PyAudio()

    # Set up recording stream
    stream = audio.open(format=pyaudio.paInt16, channels=1,
                        rate=sample_rate, input=True,
                        frames_per_buffer=chunk)

    print("Recording...")
    frames = []

    for _ in range(0, int(sample_rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    print("Recording finished.")
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save audio file
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(frames))

if __name__ == "__main__":
    person_name = input("Enter the name of the person: ")
    num_samples = 5
    dataset_dir = f"voice_dataset/{person_name}"
    
    if not os.path.exists(dataset_dir):
        os.makedirs(dataset_dir)

    for i in range(num_samples):
        filename = os.path.join(dataset_dir, f"{i+1}.wav")
        record_audio(filename, duration=5)


Recording...
Recording finished.
Recording...
Recording finished.
Recording...
Recording finished.
Recording...
Recording finished.
Recording...
Recording finished.
Recording...
Recording finished.
Recording...
Recording finished.
Recording...
Recording finished.
Recording...
Recording finished.
Recording...
Recording finished.


In [3]:
import librosa
import numpy as np
from sklearn.svm import SVC
import joblib
import os

def extract_features(filename):
    y, sr = librosa.load(filename)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs, axis=1)

def train_model(dataset_folder):
    features = []
    labels = []
    label_dict = {}

    for filename in os.listdir(dataset_folder):
        if filename.endswith('.wav'):
            person_name = filename.split('_')[0]
            if person_name not in label_dict:
                label_dict[person_name] = len(label_dict)
            label = label_dict[person_name]
            feature = extract_features(os.path.join(dataset_folder, filename))
            features.append(feature)
            labels.append(label)

    features = np.array(features)
    labels = np.array(labels)

    model = SVC()
    model.fit(features, labels)
    joblib.dump(model, 'voice_model.pkl')
    print(f"Model trained and saved as 'voice_model.pkl'")

if __name__ == "__main__":
    dataset_folder = input("Enter the path to the dataset folder: ")
    train_model(dataset_folder)


ValueError: The number of classes has to be greater than one; got 1 class